In [2]:
import numpy as np
import sys
import copy
import networkx as nx
from pyvis.network import Network
import plotly
import plotly.graph_objects as go
import math
from timeit import default_timer as timer
import time

%matplotlib inline

In [3]:
#Settings:

QUIET = True #Just here so you can tell a function "QUIET" or "INVIS" and for some of these it will suppress part 
INVIS = False #of the output. For INVIS this will be graphical outputs
FORCE_VALID_GRIDS = False #Not fully implemented - may let some X and O be placed in the same square
FORCE_VALID_MOVES = False #Not fully implemented - prevents stabilizing where we can't and commuting columns illegally


In [4]:
class perm: #takes a list of integers and makes it a permutation type with standard permutation
            #group operation. Will copy argument if given a perm instead of a list datatype.
        
    def __init__(self, lst = [1]): #initialization just loads the list into the self.value
        check_if_valid(lst)
        if type(lst) == perm:
            self.value = lst.value.copy()
        else:
            self.value = lst.copy()

    def __str__(self):
        return str(self.value)
            
    def __repr__(self):
        return str(self.value)

    def __getitem__(self, i):
        return self.value[i-1]
    
    def __setitem__(self, i, val):
        self.value[i-1] = val
    
    def __mul__(self, other): #self o other as permutation composition, also loads this into the * notation so sig*phi works
        temp_sig = self.value.copy()
        temp_phi = other.value.copy()
        temp_sig, temp_phi = match_sizes(temp_sig, temp_phi)
        n = len(temp_phi)
        result = temp_sig.copy()
        for i in range(n):
            result[i] = temp_sig[temp_phi[i]-1]
        return perm(result)
    
    def __len__(self):
        return len(self.value)
    
    def __eq__(self, other): #lets us compare permutations
        if type(other) != perm:
            return False
        if self.value == other.value:
            return True
        else:
            return False

    def copy(self):
        newCopy = perm(self.value)
        return newCopy
        
    def inverse(self): #Finding sig^-1 
        result = identity_perm(len(self.value))
        for i in range(1,len(self.value)+1):
            result[self[i]] = i
        return result
        
    def __pow__(self, n): #computes sig**n and loads it into the ** notation
        temp_sig, temp_id = match_sizes(self.value.copy(), [1])
        result = perm(temp_id)
        if n >= 0:
            for i in range(n):
                result = self * result
            return result
        else:
            inv = self.inverse()
            for i in range(-n):
                result = inv * result
            return result

    def size(self):
        return len(self.value)
    
    def cycles(self):         #returns a tuple of tuples representing the cycle notation for
        cycle_collection = [] #the given permutation for example [2, 1, 3, 5, 6, 4] --> ((1, 2)(3)(4,5,6)
        counted = []
        for x in range(self.size()):
            current = x + 1
            current_cycle = []
            while current not in counted:
                current_cycle.append(current)
                counted.append(current)
                current = self.value[current - 1]
            if len(current_cycle) > 0:
                cycle_collection.append(tuple(current_cycle))
        return tuple(cycle_collection)
    
    def reduce_at(self, position): #removes an element that maps to itself and renumbers the permutation so its consistent
        if self.value[position - 1] == position:
            result = []
            for x in self.value:
                if x < position:
                    result.append(x)
                elif x > position:
                    result.append(x-1)
            return perm(result)
        else:
            raise ValueError("Cannot reduce at given value - given value is not fixed under the permutation")
                    
    def collapse_at(self, position): #pass this function the i s.t. you want to remove sigma(i)
        original_value = self.value[position - 1]
        result = []
        reference = self.value.copy()
        for x in reference:
            if x < original_value:
                result.append(x)
            elif x > original_value:
                result.append(x - 1)
        return perm(result)
    
    def widen_at(self, position = 1, value = 1):  #inverse of the reduce function - widens a permutation by adding i -> i in
        reference = self.value.copy()  #the middle of a permutation and renumbers the inputs and outputs that are
        size = len(reference)+1        # larger than i accordingly.
        reference.append(size)         # Ex: [1, 3, 4, 6, 2, 5] widen at 4 --> [1, 3, 5, 4, 7, 2, 6]
        result = []                    #This operation isn't very natural in permutations but it is in the context of grids
        for i in range(size):
            if i+1 < position:
                if reference[i] < value:
                    result.append(reference[i])
                else:
                    result.append(reference[i]+1)
            elif i+1 > position:
                if reference[i-1] < value:
                    result.append(reference[i-1])
                else:
                    result.append(reference[i-1]+1)                
            else:
                result.append(value)
        return perm(result)

def identity_perm(n): #produces perm data type of [1, 2, 3 ... n]
    temp_list = []
    for i in range(n):
        temp_list.append(i+1)
    return perm(temp_list)

def extend_perm(sig, n, isPerm = False): #Extends a permutation to length n - will not shorten a given permutation
    check_if_valid(sig)                  #accepts lists and perms returning the same type as given
    if type(sig) == list:
        size = len(sig)
        if n <= size:
            if isPerm:
                return perm(sig)
            return sig
        extended_perm = sig
        for i in range(size+1, n+1):
            extended_perm.append(i)
        if isPerm:
            return perm(extended_perm)
        return extended_perm
    else:
        return extend_perm(sig.value, n, True)

def match_sizes(sigma, phi): #extends the sigma and phi as necessary, adding elements mapping to themselves
    check_if_valid(sigma)    #accepts lists and perms returning the same type as given
    check_if_valid(phi)
    new_sigma = perm(sigma)
    new_phi = perm(phi)
    fin_sigma = extend_perm(new_sigma, len(phi))
    fin_phi = extend_perm(new_phi, len(sigma))
    return (type(sigma)(fin_sigma.value), type(phi)(fin_phi.value))

def check_if_valid(sig): #Running collection of possible errors for given permutation arguments
    if not ((type(sig) == list) or (type(sig) == perm)): #checks if sig is a list or a perm type and raises a warning if not
        raise ValueError("Neither list nor perm passed to function")
        
def transposition(x, y, n = 2): #Permutation of [1,2 ... y ... x ... n] just swapping x and y
    temp_list = []
    for i in range(n):
        temp_list.append(i+1)  
    temp_list[x-1] = y
    temp_list[y-1] = x
    return perm(temp_list)

def generate_all_transpositions(n):
    temp_list = []
    for x in range(1,n):
        for y in range(x+1,n+1):
            temp_list.append(transposition(x,y,n))
#            print("transposing x = " + str(x) + " and y = " + str(y))
    return temp_list

def generate_all_labeled_transpositions(n):
    temp_list = []
    for x in range(1,n):
        for y in range(x+1,n+1):
            temp_list.append([transposition(x,y,n), [x,y]])
    return temp_list

def generate_sn(n):
    ref = list(range(1,n+1))
    temp_list = []
    working_x = []
    hold = []
    for x in range(1,n+1):
        temp_list.append([x])
    for i in range(n-1):
#        print(i)
        for z in range(1,n+1):
            for x in temp_list:
                if z not in x:
                    working_x = x.copy()
                    working_x.append(z)
                    hold.append(working_x)
        temp_list = hold.copy()
        hold = []
    result = []
    for entry in temp_list:
        result.append(perm(entry))
    return result

        
def full_cycle(n): #permutation of [1, 2, 3, ... n]
    result = []
    for i in range(n-1):
        result.append(i+2)
    result.append(1)
    return perm(result)

def perm_from_cycle(cycle, given_size = -1): #takes a TUPLE of TUPLES so make sure to include commas if either of those tuples has a single element
    size = 0              #currently broken - doesn't handled duplicate instances of an element -- requires reduced cycles
    if given_size == -1:
        for sub_cycle in cycle:
            for x in sub_cycle:
                if x > size:
                    size = x
    else:
        size = given_size
    temp_sig = []
    for i in range(size):
        temp_sig.append(i+1)
    for sub_cycle in cycle:
        for i in range(len(sub_cycle)):
            temp_sig[sub_cycle[i-1]-1] = sub_cycle[i]
    return perm(temp_sig)



In [5]:
import matplotlib.pyplot as plt  #code necessary for the grid diagram printing written by Evan Bell from SURIEM 2021 - saving here in case
import itertools                 #I decide to duplicate the functionality here
from scipy.optimize import curve_fit
from collections import defaultdict
import networkx as nx
from numba import jit
from numba import njit
%matplotlib inline

# Function to create a grid diagram given permutations of X and O positions
def display_grid(x_list, o_list):
    # Getting the size of the grid
    if len(x_list) == len(o_list):
        n = len(x_list)
    else:
        print('The length of the index lists do not match.')
    
    # Initializing figure and making gridlines
    plt.figure(figsize=(10*n/6,10*n/6))
    plt.vlines(range(2,n+1), ymin=1, ymax=n+1, zorder=8, color='gray', linewidth=3)
    plt.hlines(range(2,n+1), xmin=1, xmax=n+1, zorder=10.5, color='gray', linewidth=3)
    
    # Placing the X's and O's
    plt.scatter(np.arange(1, n+1)+0.5, np.array(x_list)+0.5, marker='x', s=1500, linewidth=5, c='black')
    plt.scatter(np.arange(1, n+1)+0.5, np.array(o_list)+0.5, marker='o', s=2000, linewidth=5, c='black')
    plt.scatter(np.arange(1, n+1)+0.5, np.array(o_list)+0.5, marker='o', s=1200, linewidth=5, c='white')

    # Drawing arrows
    for x, y, i in zip(x_list, o_list, range(1, n+1)):
        if x>y: # Downward pointing arrows
            plt.arrow(i+0.5, x+0.25, 0, y-x+.6, width=0.05, head_width=0.2, color='black', length_includes_head=True, zorder=11)
            plt.arrow(i+0.5, x+0.25, 0, y-x+.6, width=0.15, head_width=0.2, color='white', length_includes_head=True, zorder=10)
        if x<y: # Upward pointing arrows
            plt.arrow(i+0.5, x+0.75, 0, y-x-0.6, width=0.05, head_width=0.2, color='black', length_includes_head=True, zorder=11)
            plt.arrow(i+0.5, x+0.75, 0, y-x-0.6, width=0.15, head_width=0.2, color='white', length_includes_head=True, zorder=10)
        if o_list.index(i)<x_list.index(i): # Rightward pointing arrows
            plt.arrow(o_list.index(i)+1.85, i+0.5, x_list.index(i)-o_list.index(i)-0.55, 0, width=0.05, head_width=0.2, color='black', length_includes_head=True, zorder=9)
        if o_list.index(i)>x_list.index(i): # Leftward pointing arrows
            plt.arrow(o_list.index(i)+1.15, i+0.5, x_list.index(i)-o_list.index(i)+0.55, 0, width=0.05, head_width=0.2, color='black', length_includes_head=True, zorder=9)
    
    # Setting up the rest of the plot
    plt.xlim(1, n+1)
    plt.ylim(1, n+1)
    plt.tick_params(axis='x', which='both', bottom=False, labelbottom=False) # https://stackoverflow.com/questions/12998430/remove-xticks-in-a-matplotlib-plot
    plt.tick_params(axis='y', which='both', left=False, labelleft=False)
      

In [6]:
STAB_DIRS = ["NE","NW","SW","SE"]

class grid:      #Carries knot data - functions asking for a knot (unless otherwise specified)  
                 #are asking for a grid
        
    def __init__(self, given_x, given_o, check_grid_conditions = FORCE_VALID_GRIDS): #Loads grid information into the object and confirms the grid size is square
        check_if_valid(given_x)
        check_if_valid(given_o)
        self.sig_x = perm(given_x)
        self.sig_o = perm(given_o)
        self.size = perm(given_x).size()
        if self.size != perm(given_o).size():
            raise ValueError("Grid dimensions are not equal to eachother")
        if check_grid_conditions:
            for i in self.size:
                if self.sig_x.value[i] == self.sig_o.value[i]:
                    raise ValueError("Invalid grid - X and O appear in the same square")
                
    def __repr__(self, hide=INVIS):
        if not QUIET:
            print("sig_x: " + str(self.sig_x))
            print("sig_o: " + str(self.sig_o))
        if not hide:
            display_grid(self.sig_x.value, self.sig_o.value)
        return str([self.sig_x,self.sig_o])

    def show(self):
        display_grid(self.sig_x.value, self.sig_o.value)
        return
    
    def __str__(self):
        return (str(self.sig_x)+str(self.sig_o))
    
    def h_cycle(self, n = 1): #cycles the grid as a toroidal diagram horizontally n (negative to the left) times
        cycler = full_cycle(self.size)
        print(cycler)
        new_sig_x = self.sig_x*((cycler)**n)
        new_sig_o = self.sig_o*((cycler)**n)
        return grid(new_sig_x, new_sig_o)
    
    def v_cycle(self, n = 1): #cycles the grid as a toroidal diagram vertically n (negative down) times
        cycler = full_cycle(n)
        new_sig_x = ((cycler)**n)*self.sig_x
        new_sig_o = ((cycler)**n)*self.sig_o
        return grid(new_sig_x, new_sig_o)
        
    def x_stab(self, x_choice, direction = "NW"): #grid stabilization at an x - essentially splits that x into two rows and columns
                                                  #this makes for 4 (valid) ways to fill in the gaps, NW,NE,SE,SW
        h_switch = perm_from_cycle(((x_choice, x_choice+1),), self.size+1)
        v_switch = perm_from_cycle(((self.sig_x[x_choice],self.sig_x[(x_choice)]+1),), self.size+1)
        temp_sig_x = self.sig_x.widen_at(x_choice, self.sig_x[(x_choice)])
        temp_sig_o = self.sig_o.widen_at(x_choice, self.sig_x[(x_choice)])
        if direction == "NW":
            temp_sig_o = temp_sig_o*h_switch
            return grid(temp_sig_x,temp_sig_o)
        if direction == "NE":
            temp_sig_x = temp_sig_x*h_switch
            return grid(temp_sig_x,temp_sig_o)
        if direction == "SE":
            temp_sig_x = temp_sig_x
            temp_sig_o = v_switch*temp_sig_o
            return grid(temp_sig_x,temp_sig_o)
        if direction == "SW":
            temp_sig_x = temp_sig_x*h_switch
            temp_sig_o = v_switch*(temp_sig_o*h_switch)
            return grid(temp_sig_x,temp_sig_o)

    def x_destab(self, x_choice, check_if_legal = FORCE_VALID_MOVES): #still need to work out how to check for legality - ie see if the place
        height = self.sig_x[x_choice]                                 #we destabilize at is one of NE NW SE SW.
        x_temp = self.sig_x.collapse_at(x_choice)
        helper = perm_from_cycle(((height,self.sig_o[x_choice]),),self.size)
        o_temp = self.sig_o
        o_temp = (helper*o_temp).collapse_at(x_choice)
        return grid(x_temp, o_temp)

    def __hash__(self):
        identifier = ''
        for x in self.sig_x:
            identifier = identifier + str(x)
        for o in self.sig_o:
            identifier = identifier + str(o)
        return int(identifier)    
    
    
    def commute_columns(self, left_position, ValidGridMove = FORCE_VALID_MOVES):
        h_switch = perm_from_cycle(((left_position, left_position+1),), self.size)
        new_sig_x = self.sig_x*h_switch
        new_sig_o = self.sig_o*h_switch
        return grid(new_sig_x, new_sig_o)
        
    def __eq__(self, other):
        if type(other) != grid:
            return False
        if self.sig_x == other.sig_x:
            if self.sig_o == other.sig_o:
                return True
        return False

#END OF GRID TYPE DEFINITION    
    
    
def generate_all_grids(n):     #this is (as far as I can tell) tied to derangements, which is an NP-complete problem so this won't be
    perm_set = generate_sn(n)  #able to be incredibly optimized
    hold = []
    for element in perm_set:
        for other in perm_set:
            sudokurule = True
            for i in range(n):
                if element[i+1] == other[i+1]:
                    sudokurule = False
                    break
            if sudokurule:
                hold.append(grid(element,other))
    return hold



In [7]:
def symbol_coordinates(symbol_perm):
    
    if type(symbol_perm) == perm:
        
        temp = symbol_perm.value.copy()
        
    else:
        
        temp = symbol_perm.copy()
        
    for count, element in enumerate(temp):
        
        temp[count] = (count + 1.5, element + 0.5)
        
    return temp


def is_between(target, a, b):
    
    if target > a:
        if target < b:
            return True
    return False


def check_case_1(rect, target): #standard position rectangle to check rect[0] has both coordinates less
                                #than rect[1]
    if (is_between(target[0], rect[0][0], rect[1][0]) and is_between(target[1], rect[0][1], rect[1][1])):
        return True
    return False

def check_case_2(rect, target): #standard position rectangle to check rect[0] has both coordinates less
                                #than rect[1]
    if ((not is_between(target[0], rect[1][0], rect[0][0])) and is_between(target[1], rect[0][1], rect[1][1])):
        return True
    return False
    
def check_case_3(rect, target): #standard position rectangle to check rect[0] has both coordinates less
                                #than rect[1]
    if (is_between(target[0], rect[0][0], rect[1][0]) and (not is_between(target[1], rect[1][1], rect[0][1]))):
        return True
    return False

def check_case_4(rect, target): #standard position rectangle to check rect[0] has both coordinates less
                                #than rect[1]
    if ((not is_between(target[0], rect[1][0], rect[0][0])) and (not is_between(target[1], rect[1][1], rect[0][1]))):
        return True
    return False
    
def parent_check(rect, target):
    ax = rect[0][0]
    ay = rect[0][1]
    bx = rect[1][0]
    by = rect[1][1]
    if ( (ax < bx) and (ay < by) ):
        return check_case_1(rect, target)
    if ( (ax > bx) and (ay < by) ):
        return check_case_2(rect, target)
    if ( (ax < bx) and (ay > by) ):
        return check_case_3(rect, target)
    if ( (ax > bx) and (ay > by) ):
        return check_case_4(rect, target)
    print("invalid rectangle given")


        
        

In [8]:
def hv_set_shift(h,v,sigs): #shifts all permutations horizontally and vertically by h and v
    result = []
    if len(sigs) == 0:
        n = 0
    else:
        n = len(sigs[0])
    hshift = full_cycle(n)**(-h)
    vshift = full_cycle(n)**v
    for sig in sigs:
#        print(type(sig))
        result.append((vshift*sig)*hshift)
    return result

def truncated_sn(n, trunc_length): #for when you want the first "trunc_length" possibilities of s_n
    pre_result = generate_sn(n)
    result = []
    for sig in pre_result:
        temp = sig.value.copy()
        trunc_sig = temp[0:trunc_length]
        if trunc_sig not in result:
            result.append(trunc_sig.copy())
    return result

def generate_all_states_outside_rectangle(rectangle, n): #rectangle given as lower left,upper right this function returns a list of states that 
#    print('rectangle given: ' + str(rectangle))
    r_width = (rectangle[1][0] - rectangle[0][0]) % n
#    print("r_width = " + str(r_width)) #debug line
    r_height = (rectangle[1][1] - rectangle[0][1]) % n
#    print("r_height = " + str(r_height)) #debug line
    if (r_width + r_height) > n:
        print("rectangle dimensions too large to have non-empty set of states")
        return []
    pre_result = []
    sa = truncated_sn(n-r_height-1, r_width - 1)    #region a is the one above the rectangle and b is the columns after that then s_a is the collection of intersection choices
#    print("populated sa as " +str(sa))  #for a grid state x that keeps the rectangle empty
    sb = generate_sn(n-abs(r_width)-1)  #sanity check: 2 intersections used by corners, rw - 1 for a and n - rw - 1 for b means 2 + a + b = n 
#    print("populated sb")
    if sa == []:
        whats_left = []
        for i in range(2,r_height+1):
            whats_left.append(i)
        for i in range(r_height+1,n+1):
            whats_left.append(i)
        for psi in sb:
            curr_state=[]
            curr_state.append(1)
            curr_state = [1+r_height] + curr_state
            for i in range(n-abs(r_width)-1):
                curr_state.append(whats_left[psi[i]-1])
            pcurr_state = perm(curr_state.copy())
#            print(pcurr_state)
            pre_result.append(pcurr_state)
    for sig in sa:
        x = sig.copy() #we're lifting this as a list rather than a permutation because we need to shift them all up by the height - making it no longer a perm
        for count, position in enumerate(x):
            x[count] = position + r_height + 1
#        print("x is " + str(x))
        whats_left = []
        for i in range(2,abs(r_height)+1):
            whats_left.append(i)
        for i in range(abs(r_height) + 2, n + 1):
            if i not in x:
                whats_left.append(i)
#        print("whats left is: " + str(whats_left))
        if sb == []:
            curr_state = x.copy()
            curr_state.append(1)
            curr_state = [1+r_height] + curr_state
            pcurr_state = perm(curr_state.copy())
            pre_result.append(pcurr_state)
        for psi in sb:
            curr_state=x.copy()
            curr_state.append(1)
            curr_state = [1+r_height] + curr_state
            for i in range(n-abs(r_width)-1):
                curr_state.append(whats_left[psi[i]-1])
            pcurr_state = perm(curr_state.copy())
#            print(pcurr_state)
            pre_result.append(pcurr_state)
#    print(str(pre_result))
    if not ((rectangle[0][0] == 1) and (rectangle[0][1] == 1)):
        raw_result = hv_set_shift(rectangle[0][0]-1, rectangle[0][1]-1, pre_result)
    else:
        raw_result = pre_result
    result = []
    rect_pairer = transposition(rectangle[0][0],rectangle[1][0],n)
    for sig in raw_result:
        result.append([sig*rect_pairer,sig])
    if result == []:
        return None
    return result

def generate_all_rectangle_sizes(n):
    result = []
    for width in range(1,n):
        for height in range(1,n+1-width):
            result.append((width,height))
    return result

def generate_all_rectangles(n):
    sizes = generate_all_rectangle_sizes(n)
    rects = []
    for size in sizes:
        for i in range(0,n):
            for j in range(0,n):
                x = (i % n + 1,j % n + 1)
                y = ((i+size[0]) % n + 1,(j+size[1]) % n + 1)
                rects.append((x,y))
#    print(rects)
    return rects

def zero_list(n):
    result = []
    for i in range(n):
        result.append(0)
    return result

def count_symbols(n, rect, symbol_perm):
    temp = zero_list(n)
    usable_sym = symbol_coordinates(symbol_perm)
    for i in range(n):
        if parent_check(rect, usable_sym[i]):
            temp[i] = 1
    return temp
        

def generate_all_edges(n, symbols): #n is the grid size and symbols are a collection of basepoints - in the case of a grid diagram this should be a list of two permutations
    pre_diff = generate_all_rectangles(n) #symbols is a list data type containing either lists or perms either should work
    symbol_count = len(symbols)
    place_holder = []
    z_list = zero_list(n)
    for i in range(symbol_count):
        place_holder.append(z_list.copy())
#    print('placeholder looks like: ' + str(place_holder))
    unweighted_diff = []
    for rect in pre_diff:
#        print('processing rectangle ' + str(rect))
        candidates = generate_all_states_outside_rectangle(rect, n)
        if candidates is not None:
            for count, candidate in enumerate(candidates):
                candidates[count] = [candidate.copy(), rect, place_holder.copy()]
            unweighted_diff = unweighted_diff + candidates
    for count, symbol in enumerate(symbols):
        for i in range(len(unweighted_diff)):
#            print(unweighted_diff)
            unweighted_diff[i][2][count] = count_symbols(n, unweighted_diff[i][1], symbol) #after this it should be weighted
    return unweighted_diff

In [9]:
generate_all_states_outside_rectangle(((1,1),(2,2)),5)

[[[1, 2, 3, 5, 4], [2, 1, 3, 5, 4]],
 [[1, 2, 3, 4, 5], [2, 1, 3, 4, 5]],
 [[1, 2, 4, 5, 3], [2, 1, 4, 5, 3]],
 [[1, 2, 4, 3, 5], [2, 1, 4, 3, 5]],
 [[1, 2, 5, 4, 3], [2, 1, 5, 4, 3]],
 [[1, 2, 5, 3, 4], [2, 1, 5, 3, 4]]]

In [15]:
comp = generate_all_edges(4,[[1,2,3,4],[2,3,4,1]])
print(comp[0])
np.shape(comp)

[[[1, 2, 3, 4], [2, 1, 3, 4]], ((1, 1), (2, 2)), [[1, 0, 0, 0], [0, 0, 0, 0]]]


(176, 3, 2)

In [ ]:
for i in range(3, 9):
    current_complex = generate_all_edges(i,[])
    with open("edgeComplex" + str(i) +".txt", 'w') as f:
        start = timer()
        for edge in current_complex:
            f.write(str(edge) + '\n')
        f.close()
        end = timer()
        print("Time for complex for size " + str(i) + " = " + str(end - start))

Time for complex for size 3 = 0.0005510000046342611
Time for complex for size 4 = 0.0008495999936712906
Time for complex for size 5 = 0.006056199999875389
Time for complex for size 6 = 0.04381499999726657
Time for complex for size 7 = 0.3740395000058925
Time for complex for size 8 = 3.9382540000078734


In [124]:
len(wTrefoilComplex)

27

In [ ]:
generate_all_rectangles(3)

[((1, 1), (2, 2)),
 ((1, 2), (2, 3)),
 ((1, 3), (2, 1)),
 ((2, 1), (3, 2)),
 ((2, 2), (3, 3)),
 ((2, 3), (3, 1)),
 ((3, 1), (1, 2)),
 ((3, 2), (1, 3)),
 ((3, 3), (1, 1)),
 ((1, 1), (2, 3)),
 ((1, 2), (2, 1)),
 ((1, 3), (2, 2)),
 ((2, 1), (3, 3)),
 ((2, 2), (3, 1)),
 ((2, 3), (3, 2)),
 ((3, 1), (1, 3)),
 ((3, 2), (1, 1)),
 ((3, 3), (1, 2)),
 ((1, 1), (3, 2)),
 ((1, 2), (3, 3)),
 ((1, 3), (3, 1)),
 ((2, 1), (1, 2)),
 ((2, 2), (1, 3)),
 ((2, 3), (1, 1)),
 ((3, 1), (2, 2)),
 ((3, 2), (2, 3)),
 ((3, 3), (2, 1))]

In [ ]:
sig = perm([1,5,2,4,3,6,8,7])
phi = transposition(2,4,8)
print(sig*phi)
print(phi*sig)

In [ ]:
generate_all_states_outside_rectangle([(1,1),(4,3)],5)

In [ ]:
generate_all_states_outside_rectangle([(1,1),(2,3)],5)

In [ ]:
generate_all_states_outside_rectangle([(2,3),(5,6)],8)

In [ ]:
#naming some common knots
trefoil = grid([5, 1, 2, 3, 4], [2, 3, 4, 5, 1])
unknot = grid([1,2],[2,1])
figureEight = grid([5,4,1,6,2,3],[2,6,5,3,4,1])
conway = grid([1,2,4,8,5,7,6,10,3,9,11],[4,5,10,3,1,11,9,8,7,2,6])


In [ ]:
# def Grid_Chain_Complex(knot):
#     n = knot.size
#     gridstate_set = generate_sn(n)
#     tempvset = []
#     for element in gridstate_set:
#         tempvset.append(str(element))
#     transpo_set = generate_all_labeled_transpositions(n)
#     G = nx.DiGraph()
#     for i in range(n):
#         G.add_node(tempvset[i]) #state=gridstate_set[i])
#     for x in gridstate_set:
#         for y in transpo_set:
#             current_rectangle = [[y[1][0],x[y[1][0]]],[y[1][1],x[y[1][1]]]]
# #            print(str(current_rectangle))
#             G.add_edge(str(x),str(x*y[0])) ###rectangle = current_rectangle###)
#     return G

In [ ]:
# G = Grid_Chain_Complex(trefoil)
# nx.write_gml(G, "demoGCC.gml")

In [ ]:
# def find_pre_rectangles(sig, psi):
#     temp = []
#     for i in range(len(sig)):
#         if sig[i+1] != psi[i+1]:
#             temp.append([i+1, sig[i+1]])
#     if len(temp) != 2:
#         print("Given sig and psi do not differ by a transposition")
#         print(temp)
#         return None
#     else:
#         return temp

In [ ]:
# def check_if_empty(rect, target): #rect is a list [[x0,y0],[x1,y1]] with data shaped like (i, sig(i)) as the coordinates of the two
#                           #corners of the rectangle
#     if type(target) == perm:
#         target = target.value.copy()
#     given_lower = rect[0][1]
#     given_upper = rect[1][1]
#     if given_lower < given_upper:
#         tracker = -1
#         lower = given_lower
#         upper = given_upper
#     else:
#         tracker = 1
#         upper = given_lower
#         lower = given_upper
#     result = [True, True]
#     n = len(target)
#     for i in range(rect[0][0]-1):
#         if check_step(target[i], lower, upper, tracker):
#             result[1] = False
#     tracker = tracker*(-1)
#     for i in range(rect[0][0]-1,rect[1][0]-1):
#         if check_step(target[i], lower, upper, tracker):
#             result[0] = False
#     tracker = tracker*(-1)
#     for i in range(rect[1][0]-1,n):
#         if check_step(target[i], lower, upper, tracker):
#             result[1] = False
#     return result

# def check_step(value, low, high, location=1): #location =+-1 plus is between -1 is outside
#     if location == 1:
#         result = ((value < high) and (value > low))
#     elif location == -1:
#         result = ((value > high) or (value < low))
#     else: #this condition should only come up when trying to check if a corner of the rectangle is in said rectangle
#         return False
#     return result

In [ ]:
def stDiff(givenGrid):
    diff = {}
    for x in range(1, givenGrid.size + 1):
        current = givenGrid.x_destab(x)
        if current in diff:
            diff[current] += 1
        else:
            diff[current] = 1
            print(current.sig_x)
            print(current.sig_o)
            display_grid(current.sig_x.value, current.sig_o.value)
    return diff

def addToGraphFromDictZ2(source, data, G):
    for target in data:
        if data[target]%2 == 1:
            G.add_edge(str(source), str(target))
    return

def stComplex(givenGrid):
    G = nx.DiGraph()
    currentSize = givenGrid.size
    curBasis = [givenGrid]
    for count in range(currentSize - 1):
        nextBasis = []
        for obj in curBasis:
            curDiff = stDiff(obj)
            addToGraphFromDictZ2(obj, curDiff, G)
            for dx in curDiff:
                print("dx is " + str(dx))
                if dx not in nextBasis:
                    nextBasis.append(dx)
        curBasis = nextBasis
    return G

# def convertNPMat(givenMatrix, desiredField):
#     result = givenMatrix.copy()
#     for x in result:
#         x = type(desiredField)(x)
#     return result

# class Zn:
    
#     def __init__(self, value, n=2):
#         self.value = value % n
#         self.modulus = n
        
#     def __add__(self, other):
#         if type(other) == Zn:
#             return Zn((other.value + self.value), self.modulus)
#         return Zn(other + self.value, self.modulus)
    
#     def __mult__(self, other):
#         return(Zn(self.value * other, self.modulus))

In [ ]:
for x in range(5):
    G = stComplex(trefoil.h_cycle(x+1))
    adMat = nx.adjacency_matrix(G)
    
    nx.write_gml(G, "stDiffTrefoil" + str(x) + ".gml")

In [ ]:
# dTrefoil = stDiff(trefoil)
# G = nx.DiGraph()
# addToGraphFromDictZ2(trefoil, dTrefoil, G)
# nx.write_gml(G, "stDiffTest.gml")

In [30]:
sig = perm([1,2,3])

In [31]:
for x in sig:
    print(x)

3
1
2
3
